In [10]:
import os
from dotenv import load_dotenv
import asyncio

from autogen_agentchat.agents import AssistantAgent,SocietyOfMindAgent,UserProxyAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat,SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


# Load environment variables from a .env file
load_dotenv("/home/azureuser/azure-ai-agent-workshop/.env")

True

In [11]:
model_client=AzureOpenAIChatCompletionClient(
            model=os.getenv("AZURE_OPENAI_MODEL"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_version=os.getenv("AZUER_OPENAI_API_VERSION")
        )

In [12]:

# define Agent
Hailing_Intent_Recognition_Agent = AssistantAgent(
    name="ride_hailing_intent_recognition_agent",
    model_client=model_client,
    description="ride-hailing intent recognition agent.",
    system_message=''' You will engage in a conversation with the user. You should first determine the user's intent. There are three possible intents: hail a car now, schedule a car, or other. If the intent is other, you do not need to do anything. If it is hail a car now or schedule a car, you need to identify the key information in the sentence and check whether the provided information is sufficient to fulfill the user's intent. If it is not sufficient, ask the user for more information one by one.

    The required information for each need is as follows:

    Hail a car now:
    1. Pickup location
    2. Destination
    3. Rider (default is the user)
    4. Car type
    5. Pickup time (default is now)

    Schedule a car:
    1. Pickup location
    2. Destination
    3. Rider (default is the user)
    4. Car type
    5. Pickup time

    When the intent is hail a car now:
    First, how to determine the pickup location, if the user's instruction already includes the specific pickup location and the city information of the pickup location, use the pickup location specified in the instruction (including the specific pickup location and the city of the pickup location); if the user's instruction already includes the specific pickup location but does not include the city information of the pickup location, further ask for the city of the pickup location to ensure that the pickup location includes both the specific pickup location and the city of the pickup location; if the user's instruction does not include the pickup location, ask the user for the specific pickup location and the city of the pickup location to ensure that the pickup location includes both the specific pickup location and the city of the pickup location.
    Second, how to determine the destination, if the user's instruction already includes the specific destination and the city information of the destination, use the destination specified in the instruction (including the specific destination and the city of the destination); if the user's instruction already includes the specific destination but does not include the city information of the destination, further ask for the city of the destination to ensure that the destination includes both the specific destination and the city of the destination; if the user's instruction does not include the destination, ask the user for the specific destination and the city of the destination to ensure that the destination includes both the specific destination and the city of the destination.
    Third, how to determine the rider, if the user does not specify a rider, you do not need to ask the user, just set the rider as the user.
    Fourth, how to determine the car type, the choice of car type depends on the application you are using. If the destination is in mainland China, you should use DiDi Enterprise Edition. Otherwise, you should use Uber. If using DiDi Enterprise Edition, you can choose any of the seven options:
    1. Carpool(拼车)
    2. Enterprise Special Express(企业特价快车)
    3. Express(快车)
    4. Comfort Car(舒适专车)
    5. DiDi Express(滴滴特快)
    6. Six-Seater Business(六座商务)
    7. Luxury Car(豪华车)
    If using Uber, you can choose one of three options:
    1. UberXL (6 seats, but expensive)
    2. UberX (4 seats)
    3. Taxi (4 seats)
    If the user's chosen car type is not in the options, ask the user again. Show detailed information to help the user make a choice.
    Fifth, how to determine the pickup time, if the user does not specify a specific pickup time, the pickup time defaults to now.

    When the intent is schedule a car:
    First, how to determine the pickup location, if the user's instruction already includes the specific pickup location and the city information of the pickup location, use the pickup location specified in the instruction (including the specific pickup location and the city of the pickup location); if the user's instruction already includes the specific pickup location but does not include the city information of the pickup location, further ask for the city of the pickup location to ensure that the pickup location includes both the specific pickup location and the city of the pickup location; if the user's instruction does not include the pickup location, ask the user for the specific pickup location and the city of the pickup location to ensure that the pickup location includes both the specific pickup location and the city of the pickup location.
    Second, how to determine the destination, if the user's instruction already includes the specific destination and the city information of the destination, use the destination specified in the instruction (including the specific destination and the city of the destination); if the user's instruction already includes the specific destination but does not include the city information of the destination, further ask for the city of the destination to ensure that the destination includes both the specific destination and the city of the destination; if the user's instruction does not include the destination, ask the user for the specific destination and the city of the destination to ensure that the destination includes both the specific destination and the city of the destination.
    Third, how to determine the rider, if the user does not specify a rider, you do not need to ask the user, just set the rider as the user.
    Fourth, how to determine the car type, the choice of car type depends on the application you are using. If the destination is in mainland China, you should use DiDi Enterprise Edition. Otherwise, you should use Uber. If using DiDi Enterprise Edition, you can choose any of the seven options:
    1. Carpool(拼车)
    2. Enterprise Special Express(企业特价快车)
    3. Express(快车)
    4. Comfort Car(舒适专车)
    5. DiDi Express(滴滴特快)
    6. Six-Seater Business(六座商务)
    7. Luxury Car(豪华车)
    If using Uber, you can choose one of three options:
    1. UberXL (6 seats, but expensive)
    2. UberX (4 seats)
    3. Taxi (4 seats)
    If the user's chosen car type is not in the options, ask the user again. Show detailed information to help the user make a choice.
    Fifth, how to determine the pickup time, if the user's instruction already includes the pickup time and the pickup time is accurate to the minute and specifies whether it is AM or PM, use the pickup time specified in the instruction. If the user's specified pickup time is not accurate to the minute or does not indicate whether it is AM or PM, continue to ask the user to ensure the pickup time is accurate to the minute and specifies whether it is AM or PM. If the user's instruction does not include the pickup time, ask the user for the specific pickup time to ensure that the pickup time is accurate to the minute and specifies whether it is AM or PM.

    After all the information is collected, ask the user to confirm the information. After the user confirms, tell the user that the hailing process starts by a single message, then call the operating function.
    You must ask the user in Chinese.
    If you receive "Finish Operating" from the operating function and want to end the conversation, then Respond with '<FINISH>'.'''
    )



In [13]:

# define user proxy Agent
user_proxy = UserProxyAgent(
    name="user_proxy"
)

inner_termination = TextMentionTermination("<FINISH>")
inner_team = SelectorGroupChat([Hailing_Intent_Recognition_Agent, user_proxy], model_client=model_client, termination_condition=inner_termination)

society_of_mind_agent = SocietyOfMindAgent("society_of_mind", team=inner_team, model_client=model_client)

In [ ]:
from pydantic import BaseModel
import json

class Rent_Car_Info(BaseModel):
    
    # 1. Pickup location
    pickup_location: str
    # 2. Destination
    destination: str
    # 3. Rider (default is the user)
    rider: str = "user"
    # 4. Car type
    car_type: str
    # 5. Pickup time (default is now)
    pickup_time: str = "now"
    # 6. Intent
    intent: str = "Hail a car now"


# define Tool
async def call_operating(rent_Car_Info: Rent_Car_Info) -> str:
    rent_car_info_str = json.dumps(rent_Car_Info, ensure_ascii=False)
    print("call_operating rent_car_info_str:", rent_car_info_str)
    print("<叫车服务成功>")
    return " 打车信息如下：" + rent_car_info_str + "\n  <叫车服务成功>"


car_call_operating_Agent = AssistantAgent(
        "car_call_operating_Agent", 
        model_client=model_client, 
        system_message="You are a tool agent. You will receive a json string as input, which contains the information of the car rental. You need to call the operating function and return the result.",
        description="You are a tool agent. You will receive a json string as input, which contains the information of the car rental. You need to call the operating function and return the result.",
        tools=[call_operating])

task_termination = TextMentionTermination("<FINISH>")
team = RoundRobinGroupChat([society_of_mind_agent, car_call_operating_Agent], max_turns=10,termination_condition=task_termination)

stream = team.run_stream(task="我要打车去虹桥火车站.")
await Console(stream)

---------- user ----------
我要打车去虹桥火车站.


---------- ride_hailing_intent_recognition_agent ----------
请问您从哪里出发？需要提供具体的接送地点和城市信息。
---------- user_proxy ----------
我在上海，我在徐汇区虹漕路77号
---------- ride_hailing_intent_recognition_agent ----------
好的，您的接送地点是徐汇区虹漕路77号，城市是上海。请问您的目的地是哪里？
---------- user_proxy ----------
虹桥火车站
---------- ride_hailing_intent_recognition_agent ----------
请问您是否有指定的乘客？默认是您本人。还有，您希望选择什么类型的车呢？可选的类型有：
1. UberXL (6个座位，价格较贵)
2. UberX (4个座位)
3. Taxi (4个座位)

请告诉我您的选择。
---------- user_proxy ----------
3. Taxi (4个座位)
---------- ride_hailing_intent_recognition_agent ----------
好的，以下是您打车的信息：
- 接送地点：上海 徐汇区虹漕路77号
- 目的地：虹桥火车站
- 乘客：您本人
- 车的类型：Taxi (4个座位)
- 接送时间：现在

请确认以上信息是否正确？
---------- user_proxy ----------
现在
---------- ride_hailing_intent_recognition_agent ----------
谢谢您的确认。现在我将开始打车请求。请稍等。
---------- user_proxy ----------

---------- ride_hailing_intent_recognition_agent ----------
<FINISH>
---------- society_of_mind ----------
请提供接送地点和城市信息，以便我们为您安排打车服务。


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='我要打车去虹桥火车站.', type='TextMessage'), TextMessage(source='ride_hailing_intent_recognition_agent', models_usage=RequestUsage(prompt_tokens=1544, completion_tokens=20), content='请问您从哪里出发？需要提供具体的接送地点和城市信息。', type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='17912655-580f-4010-ae1d-7da10c8668a4', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='我在上海，我在徐汇区虹漕路77号', type='TextMessage'), TextMessage(source='ride_hailing_intent_recognition_agent', models_usage=RequestUsage(prompt_tokens=1596, completion_tokens=30), content='好的，您的接送地点是徐汇区虹漕路77号，城市是上海。请问您的目的地是哪里？', type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='cef427a9-278d-4211-8dd9-26cbbcffa2f0', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='虹桥火车站', type='TextMe